In [1]:
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network, load_data_to_gpu
from beamng_control import beamng_server, beamng_client
from pcdet_tools import beamng_point_cloud_process_tools, draw_scenes
from pcdet.utils import common_utils
import open3d as o3d
import numpy as np

import time

2023-06-12 10:42:09,003 |INFO     |beamngpy                      |Started BeamNGpy logging.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
logger = common_utils.create_logger()

In [3]:
server_a = beamng_server()
client_b = beamng_client(logger=logger)

In [4]:
server_a.init_server()
server_a.launch_server()

2023-06-12 10:42:11,412 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=5788, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 11308)>

2023-06-12 10:42:11,412 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=5788, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 11308)>

2023-06-12 10:42:11,415 |INFO     |beamngpy.BeamNGpy             |Opening BeamNGpy instance.
2023-06-12 10:42:11,422 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2023-06-12 10:42:21,424 |INFO     |beamngpy.BeamNGpy             |Connecting to BeamNG.tech at: (127.0.0.1, 64256)
2023-06-12 10:42:21,749 |INFO     |beamngpy.BeamNGpy             |Successfully connected to Be

In [5]:
client_b.init_client()
client_b.launch_client()

2023-06-12 10:42:22,775 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-06-12 10:42:22,776 |INFO     |beamngpy.BeamNGpy             |BeamNGpy successfully connected to existing BeamNG instance.


In [6]:
server_a.debug_luanch_test()

2023-06-12 10:42:48,064 |INFO     |beamngpy.BeamNGpy             |Loaded map.
2023-06-12 10:42:49,855 |INFO     |beamngpy.Vehicle              |Vehicle vehicleA connected to simulation.
2023-06-12 10:42:49,855 |INFO     |beamngpy.BeamNGpy             |Attempting to connect to vehicle vehicleA
2023-06-12 10:42:50,951 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-06-12 10:42:50,952 |INFO     |beamngpy.BeamNGpy             |Successfully connected to vehicle vehicleA.
2023-06-12 10:42:50,954 |INFO     |beamngpy.Scenario             |Connected to scenario: tag
2023-06-12 10:42:50,978 |INFO     |beamngpy.BeamNGpy             |Starting scenario.


{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

In [7]:
CFG_FILE = ".\\cfgs\\kitti_models\\pointpillar.yaml"
CKPT_FILE = "D:\\models_zoo\\pointpillar_7728.pth"

logger.info('-----------------Quick Demo of OpenPCDet-------------------------')
cfg = cfg_from_yaml_file(CFG_FILE, cfg)

process_data = beamng_point_cloud_process_tools(dataset_cfg = cfg.DATA_CONFIG, logger=logger, class_names=cfg.CLASS_NAMES)
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=process_data)
model.load_params_from_file(filename=CKPT_FILE, logger=logger, to_cpu=True)
model.cuda()
model.eval()

2023-06-12 10:42:50,984   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2023-06-12 10:42:51,172 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

2023-06-12 10:42:51,172 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

2023-06-12 10:42:51,277   INFO  ==> Loading pa

PointPillar(
  (vfe): PillarVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=10, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone_3d): None
  (map_to_bev_module): PointPillarScatter()
  (pfe): None
  (backbone_2d): BaseBEVBackbone(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(64, eps=0.001, moment

In [8]:
vis = o3d.visualization.Visualizer()
vis.create_window()

# scene_point_cloud = o3d.geometry.PointCloud()
# scene_point_cloud.points = o3d.utility.Vector3dVector(np.random.randn(1000, 3))
# vis.add_geometry(scene_point_cloud)


def callback(points):
    # scene_point_cloud.points = o3d.utility.Vector3dVector(np.random.randn(1000, 3))
    
    pre_time = time.perf_counter()
    data_dict = process_data.point_cloud_input(points)
    if data_dict['points'].shape[0] > 0 :
        # scene_point_cloud.points = o3d.utility.Vector3dVector(data_dict["points"][:,:3])
        data_dict = process_data.collate_batch([data_dict])
        load_data_to_gpu(data_dict)

        pred_dicts, _ = model.forward(data_dict)

        draw_scenes(vis,
                points=data_dict['points'][:, 1:], ref_boxes=pred_dicts[0]['pred_boxes'].detach(),
                ref_scores=pred_dicts[0]['pred_scores'].detach(), ref_labels=pred_dicts[0]['pred_labels'].detach(), confidence=0.5
            )
        
        # draw_scenes(vis,
        #         points=data_dict['points'][:, 1:], ref_boxes=None,
        #         ref_scores=None, ref_labels=None, confidence=0.7
        #     )


        vis.poll_events()
        vis.update_renderer()
        vis.clear_geometries()
        
        
client_b.debug_luanch_test(callback)

2023-06-12 10:42:52,112 |INFO     |beamngpy.BeamNGpy             |Attempting to connect to vehicle vehicleA
2023-06-12 10:42:52,132 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-06-12 10:42:52,133 |INFO     |beamngpy.BeamNGpy             |Successfully connected to vehicle vehicleA.
2023-06-12 10:42:52,164 |INFO     |beamngpy.Lidar                |Opened lidar: "lidar"


tag
{'vehicleA': <beamngpy.vehicle.vehicle.Vehicle object at 0x000001B2D3E66770>}


2023-06-12 10:42:52,828   INFO  FPS:1.50761344787851
2023-06-12 10:42:54,510   INFO  FPS:0.5948366159455084
2023-06-12 10:42:55,307   INFO  FPS:1.256041244242328
2023-06-12 10:42:56,225   INFO  FPS:1.0906989231577557
2023-06-12 10:42:56,941   INFO  FPS:1.396988511929881
2023-06-12 10:42:57,874   INFO  FPS:1.0724175702096215
2023-06-12 10:42:58,565   INFO  FPS:1.4490874011350716
2023-06-12 10:42:59,141   INFO  FPS:1.7399235375117512
2023-06-12 10:42:59,969   INFO  FPS:1.2083209328863647
2023-06-12 10:43:00,594   INFO  FPS:1.6028854504487922


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


2023-06-12 10:43:04,759   INFO  FPS:0.24012064429955612
2023-06-12 10:43:08,577   INFO  FPS:0.2619054490949245
2023-06-12 10:43:09,177   INFO  FPS:1.6704761875143346
2023-06-12 10:43:09,756   INFO  FPS:1.7302112710903244


...

2023-06-12 10:43:11,674   INFO  FPS:1.5606805878558916
2023-06-12 10:43:12,272   INFO  FPS:1.6731509633202042
2023-06-12 10:43:12,847   INFO  FPS:1.740229351839947
2023-06-12 10:43:13,423   INFO  FPS:1.7392232939743768
2023-06-12 10:43:14,006   INFO  FPS:1.7175534824428527
2023-06-12 10:43:14,598   INFO  FPS:1.6891754928893468
2023-06-12 10:43:15,191   INFO  FPS:1.6902544491705365
2023-06-12 10:43:15,839   INFO  FPS:1.543241312795983
2023-06-12 10:43:16,393   INFO  FPS:1.8078125342768812
2023-06-12 10:43:16,989   INFO  FPS:1.6793815440146596
2023-06-12 10:43:17,540   INFO  FPS:1.818446650077503
2023-06-12 10:43:18,138   INFO  FPS:1.674769761053037


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


2023-06-12 10:43:18,672   INFO  FPS:1.872252119186107
2023-06-12 10:43:19,272   INFO  FPS:1.6695274020144804
2023-06-12 10:43:19,790   INFO  FPS:1.9352537629864788
2023-06-12 10:43:20,407   INFO  FPS:1.6233998145717068
2023-06-12 10:43:20,958   INFO  FPS:1.8187228056630602


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


2023-06-12 10:43:21,524   INFO  FPS:1.7656872929205985
2023-06-12 10:43:22,098   INFO  FPS:1.7445753301618618
2023-06-12 10:43:22,689   INFO  FPS:1.6908457611830208
2023-06-12 10:43:23,261   INFO  FPS:1.7545592661608778
2023-06-12 10:43:23,874   INFO  FPS:1.6300715601892015


In [ ]:
client_b.disconnect_client()


In [ ]:
server_a.close_server()